<a href="https://colab.research.google.com/github/kimanuul/python_lectures/blob/main/day0701_ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
title: "선형 회귀"
date: '2022-07-01'
---

## 로지스틱 회귀
- 선형 회귀에서 출발
- 이진 분류 문제 해결
- 클래스 확률 예측
- 딥러닝에서도 사용됨

## 교재 177
- x가 사각형일 확률 30%
- x가 삼각형일 확률 50%
- x가 원일 확률 20%

## 데이터 불러오기
- Species (종속변수 -> Y)
- Weight, Length, Diagonal, Height, Width (독립변수)

In [1]:
import pandas as pd

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


## 데이터 탐색

In [2]:
# 종속변수부터 확인
print(pd.unique(fish['Species']))
print(fish['Species'].value_counts())




['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']
Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: Species, dtype: int64


## 데이터 가공

In [3]:
# pandas 데이터 프레임에서 numpy 배열로 변환

fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']]. to_numpy()
fish_input.shape

(159, 5)

In [4]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


- 타겟 데이터, 종속변수, Y

In [5]:
fish_target = fish['Species'].to_numpy()
print(fish_target.shape)
print(fish_target[:5])


(159,)
['Bream' 'Bream' 'Bream' 'Bream' 'Bream']


## 데이터 분리
- 훈련데이터 테스트 데이터 분리

In [6]:
from sklearn.model_selection import train_test_split

#임의 샘플링
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state = 42
)

#층화 샘플링



## 표준화 전처리
- 여기에서도 훈련 세트의 통계 값으로 테스트 세트를 변환해야 한다는 점을 잊지 말기
- 데이터 가공
  + 숫자 결측치가 존재, 평균값으로 대체
  + 원본 데이터 평균 대치 -> X
  + 훈련 데이터와 테스트 데이터 분리
  - 데이터 누수(Data leakage)

## 교재 97p
- 기준을 맞춰라 -> 데이터 표준화 (표준점수)
- 수동으로 mean, std

In [7]:
# train_scaled = (train_input - mean) /std

In [8]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
# ss.fit(test_input) -> X

train_scaled = ss.transform(train_input) # train_scaled = (train_input - mean) /std
test_scaled = ss.transform(test_input)

## 모형 만들기
- K-최근접 이웃

In [9]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors = 3)
kn.fit(train_scaled, train_target)

print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


- 타깃값 확인
- 알파벳 순으로 정렬

In [10]:
print(kn.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


-  다중분류

In [11]:
print(kn.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']


- 5개 샘플에 대한 예측은 어떤 확률이냐?

In [12]:
import numpy as np
proba = kn.predict_proba(test_scaled[:5])
print(kn.classes_)
print(np.round(proba, decimals = 4))

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


- 첫번째 클래스 Perch
  + 100% 확률로 Perch로 예측
- 네번째 클래스 Perch
  + 66.7% 확률로 Perch로 예측
  + 33.3% 확률로 Roach로 예측

br     park  perch
0.     0.     1.     0.     0.     0.     0.

## 회귀 식
$y  = ax + b$
- 양변에 로그를 취함

## 로지스틱 회귀로 이진 분류 수행 

In [13]:
char_arr = np.array(['A', 'B', 'C', 'D', 'E'])
print(char_arr[[True, False, True, False, False]])



['A' 'C']


- 도미와 빙어의 행만 골라냄 

In [14]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
print(bream_smelt_indexes)
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

train_scaled.shape, train_bream_smelt.shape

[ True False  True False False False False  True False False False  True
 False False False  True  True False False  True False  True False False
 False  True False False  True False False False False  True False False
  True  True False False False False False  True False False False False
 False  True False  True False False  True False False False  True False
 False False False False False  True False  True False False False False
 False False False False False  True False  True False False  True  True
 False False False  True False False False False False  True False False
 False  True False  True False False  True  True False False False False
 False False False False  True  True False False  True False False]


((119, 5), (33, 5))

In [15]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)


LogisticRegression()

In [16]:
# 클래스를 예측
print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


- 확률값 구하기 

In [17]:
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


In [18]:
print(lr.classes_)

['Bream' 'Smelt']


- 분류기준 : threshold 임계값 설정

## 계수와 절편

In [19]:
print(lr.coef_, lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


In [20]:
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


- z값을 확률값으로 변환

In [21]:
from scipy.special import expit
print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


## 다중 분류 수행하기

In [22]:
# 하이퍼 파라미터 세팅
# 모형을 튜닝
# 순정 쓰세요
# 모형 결과의 과대적합 또는 과소적합을 방지하기 위한 것
lr = LogisticRegression(C = 20, max_iter = 1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [23]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [24]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals = 3))
print(lr.classes_)

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


- 다중 분류일 경우 선형 방정식은 어떤 모습일까?
  + 분류 7개, 컬럼 값 5개

In [25]:
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


## 평가지표


## 회귀 평가지표
- 결정계수 (교재 121p)
  + (타깃 - 예측)^2 합 / (타깃 - 평균)^2 합
- MAE, MSE, RMSE
  + (실제 - 예측) = 오차
  + MAE(Mean Absolute Error) : 오차의 절대값의 평균
평균
  + MSE(Mean Squared Error) : 오차 제곱의 평균
  + RMSE(Root Squared Error) : MSE의 제곱근을 취한 값

- 좋은 모델이란
  + 결정계수 : 1에 수렴하면 좋은 모델
  + MAE, MSE, RMSE : 0에 수렴하면 좋은 모델


In [26]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

true = np.array([1, 2, 3, 2, 3, 5, 4, 6, 5, 6, 7, 8, 8]) # 실제값
preds = np.array([1, 1, 2, 2, 3, 4, 4, 5, 5, 7, 7, 6, 8]) # 예측값

# 절대값 오차의 평균
mae = mean_absolute_error(true, preds)
print(mae)

# 제곱 오차의 평균
mse = mean_squared_error(true, preds)
print(mse)

# 제곱근
rmse = np.sqrt(mse)
print(rmse)

# 결정계수
r2 = r2_score(true, preds)
print(r2)

0.5384615384615384
0.6923076923076923
0.8320502943378437
0.8617021276595744


### 분류 평가지표
- 오차 행렬
- 실제 값
  + [빙어, 도미, 도미, 빙어, 도미]
- 예측 값
  + [빙어, 빙어, 도미, 빙어, 빙어]

TP : 2 (빙어를 빙어로 예측)
TN : 1 (도미를 도미로 예측)
FN : 2 (빙어를 도미로 예측)
FP : 0 (도미를 빙어로 예측)

- TP, TN, FP, FN
  + 정확도 : (TP + TN) / ALL
  + 정밀도(Precision) : TP / (TP + FN)
  + 재현율 : TP / T RUE

  + 로그손실
  + ROC Curve (=AUC)

In [27]:
from sklearn.metrics import confusion_matrix

true = [0, 1, 1, 0, 0]
preds = [1, 0, 0, 0, 0]

confusion_matrix(true, preds)

array([[2, 1],
       [2, 0]])